- [Inception v1](https://cdn-images-1.medium.com/max/2560/1*ZFPOSAted10TPd3hBQU8iQ.png) 
- [Inceptionv1](https://norman3.github.io/papers/images/google_inception/f03.png)
- [Inception V3](https://cdn-images-1.medium.com/max/800/1*gqKM5V-uo2sMFFPDS84yJw.png)
- [Inception v3](https://resources.wolframcloud.com/NeuralNetRepository/resources/images/8d8/8d890763-4d74-4f89-90bf-88ef7c60f439-io-1-o.en.gif)
- [Reference](https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/)

# Inception 1 From Scratch

In [0]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10

Using TensorFlow backend.


In [0]:
from keras.models import Model
from keras.layers import Conv2D,MaxPool2D, \
                  Dropout,Dense,Flatten,Input ,\
                  concatenate,AveragePooling2D, \
                  GlobalAveragePooling2D

import cv2
import numpy as np
from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

import math
import random
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.utils import to_categorical




**Load the dataset**


In [0]:
num_classes=10
# read data
(X_train, y_train), (X_valid, y_valid) = cifar10.load_data()
print(X_train.shape) #(50000, 32, 32, 3)
print(y_train.shape) #(50000, 1)
print(X_valid.shape) #(10000, 32, 32, 3)
print(y_valid.shape) #(10000, 1)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


**Preprocessing**
 - here i have limited the data as it is crashing when using all data 

In [0]:
# limit the amount of the data
# train data
ind_train = random.sample(list(range(X_train.shape[0])), 5000)#random 1000 value positions from 50000
#print(ind_train)
#print(len(ind_train))    #1000
X_train = X_train[ind_train]
print(X_train.shape)#(1000, 32, 32, 3)
y_train = y_train[ind_train]
print(y_train.shape) #(1000, 1)


# test data
ind_test = random.sample(list(range(X_valid.shape[0])), 1000)
X_valid = X_valid[ind_test]
y_valid = y_valid[ind_test]

def resize_data(data):
    data_upscaled = np.zeros((data.shape[0], 224,224, 3))
    for i, img in enumerate(data):
        large_img = cv2.resize(img, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
        data_upscaled[i] = large_img

    return data_upscaled

# resize train and  test data
X_train = resize_data(X_train)
print(X_train.shape) #(1000, 224, 224, 3)
X_valid = resize_data(X_valid)
print(X_valid.shape) #(1000, 224, 224, 3)

# make explained variable hot-encoded
y_train = to_categorical(y_train) #(1000, 10)
y_valid = to_categorical(y_valid)


(5000, 32, 32, 3)
(5000, 1)
(5000, 224, 224, 3)
(1000, 224, 224, 3)


**Basic Deep learning Architecture for Inception V1**
- [Inception v1 basic model](https://cdn-images-1.medium.com/max/1600/1*sezFsYW1MyM9YOMa1q909A.png)
- Now define a function for deep learning architecture, so that when given the necessary information, it gives us back the entire inception layer.

In [0]:
def inception_module(x,                           #previous acivation layer
                     filters_1x1,                 # (1,1)
                     filters_3x3_reduce,          # (1,1)
                     filters_3x3,                 # (3,3)
                     filters_5x5_reduce,          # (1,1)
                     filters_5x5,                 # (5,5)
                     filters_pool_proj,           # maxpooling
                     name=None):
    #----------
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    #----------
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
    #----------
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)
    #----------
    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)
    #----------
    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

##GoogLeNet architecture : -
- To create the GoogLeNet architecture, as mentioned in the paper
- [Googlenet architecture](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/Screenshot-from-2018-10-16-11-56-41-850x462.png) 
- [inception reference image](https://norman3.github.io/papers/images/google_inception/f03.png)

In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)


In [0]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

#-----
#inception 3a
#-----
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

#-----
#inception 3b
#-----
x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

#-----
#inception 4a
#-----
x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

#-----
#inception 4b
#-----
x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')
#-----
#inception 4c
#-----

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')
#-----
#inception 4d
#-----

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

#-----
#inception 4e
#-----
x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

#-----
#inception 5a
#-----
x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

#-----
#inception 5b
#-----
x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(10, activation='softmax', name='output')(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3x3/2[0][0]           
__________________________________________________________________________________________________
conv_2b_3x


The model looks fine.We can add a few finishing touches before we train our model. We will define the following:
- Loss function for each output layer
- Weightage assigned to that output layer
- Optimization function, which is modified to include a weight decay after every 8 epochs
- Evaluation metric



In [0]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8   # learning rate can reduce or change for each 8th epoch
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [0]:
history = model.fit(X_train, [y_train, y_train, y_train], validation_data=(X_valid, [y_valid, y_valid, y_valid]), epochs=epochs, batch_size=256, callbacks=[lr_sc])

Instructions for updating:
Use tf.cast instead.
Train on 5000 samples, validate on 1000 samples
Epoch 1/50

Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
5000/5000 [==============================] - 47s 9ms/step - loss: 4.3593 - output_loss: 2.6459 - auxilliary_output_1_loss: 3.2265 - auxilliary_output_2_loss: 2.4847 - output_acc: 0.1036 - auxilliary_output_1_acc: 0.1056 - auxilliary_output_2_acc: 0.1040 - val_loss: 3.7098 - val_output_loss: 2.3269 - val_auxilliary_output_1_loss: 2.3054 - val_auxilliary_output_2_loss: 2.3042 - val_output_acc: 0.0880 - val_auxilliary_output_1_acc: 0.1030 - val_auxilliary_output_2_acc: 0.0940
Epoch 2/50

Epoch 00002: LearningRateScheduler setting learning rate to 0.01.
5000/5000 [==============================] - 21s 4ms/step - loss: 3.7179 - output_loss: 2.3352 - auxilliary_output_1_loss: 2.3049 - auxilliary_output_2_loss: 2.3043 - output_acc: 0.0950 - auxilliary_output_1_acc: 0.0962 - auxilliary_output_2_acc: 0.1068 - val_loss: 3.69